In [1]:
from sklearn import svm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from joblib import dump

def svm_model(dataSet, testData_persentage, svm_kernel_type, poly_degree):
    img_train, img_test, label_train, label_test = train_test_split(dataSet.data, dataSet.target,test_size=testData_persentage,random_state=0)
    classifier = svm.SVC(kernel = svm_kernel_type, C = 1, gamma = 'auto', degree = poly_degree)
    
    #fitting classifier model
    classifier.fit(img_train, label_train)
    
    # classification_report outputs classification metrics
    # such as precision, recall and F1 score
    pred_result = classifier.predict(img_train)
    print('Classification Training Report: \n', classification_report(label_train, pred_result))
    
    # confusion_matrix outputs how many samples are correctly or incorrectly classified
    print('Train Confusion Matrix: \n', confusion_matrix(label_train, pred_result), "\n")

    # accuracy computes classification accuracy
    print('Train Accuracy: ', accuracy_score(label_train, pred_result))
    print('\n')
    
    # testing with validate data
    validate_result = classifier.predict(img_test)
    print('Classification Testing Report: \n', classification_report(label_test, validate_result))
    # `confusion_matrix` outputs how many samples are correctly or incorrectly classified
    print('Test Confusion Matrix: \n', confusion_matrix(label_test, validate_result), "\n")
    # `accuracy` computes classification accuracy
    print('Test Accuracy: ', accuracy_score(label_test, validate_result))
    
    #export model
    dump(classifier, 'svm_model.joblib')

    # plot_images_labels_wrong_prediction(img_test, label_test, validate_result, idx = 0, num=10)

In [2]:
def plot_images_labels_wrong_prediction(images, labels, prediction, idx, num=100):
    fig = plt.gcf()
    fig.set_size_inches(50, 50)
    if num>25: num=25 
    for i in range(0, num):
        if idx < 100 :
            while labels[idx] == prediction[idx] :
                idx+=1
            else:
                ax=plt.subplot(5,5, 1+i)
                image = np.reshape(images[idx], (50, -1))
                ax.imshow(image, cmap='gray')

                ax.set_title("label=" +str(labels[idx])+
                     ",predict="+str(prediction[idx])
                     ,fontsize=30) 
        
                ax.set_xticks([]);ax.set_yticks([])        
                idx+=1 
    plt.show()

In [3]:
#lbp
import cv2
from skimage.feature import local_binary_pattern
def lbp_histogram(image, radius, bins):
    # compute the LBP histogram of the image
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp, bins= bins, density=True)
    return np.ravel(hist)

In [4]:
from sklearn import datasets
from sklearn.utils import Bunch
npdata = np.load('1776_datasetA_200_40.npz')
print(npdata['data'].shape)

(1776, 200, 200)


In [29]:
number_image = 1776
radius = 1 # radius of local binary pattern histogram
bins = 8 # max = 256 

features=np.zeros(shape=(number_image, bins)) 
#add lbp to features
for i in range(0,number_image):
     features[i] = lbp_histogram(npdata['data'][i], radius, bins)
        
#add depth as features
features = np.insert(features, bins, npdata['depth'], axis=1)
print(features.shape)

(1776, 9)


In [32]:
# normalize lbp
for i in range(number_image):
    norm = np.linalg.norm(features[i, :])
    for j in range(bins):
        features[i, j] = features[i, j] / norm
        
norm = np.linalg.norm(features[:, bins])
for i in range(number_image):
    features[i, bins] = features[i, bins] / norm
    
# collect data
dataset = Bunch(data = features, target=npdata['label'])
print(dataset.data.shape)

(1776, 9)


In [39]:
svm_model(dataset, 0.7, 'linear', 2)

Classification Training Report: 
               precision    recall  f1-score   support

           0       0.98      0.67      0.80       321
           1       0.66      0.98      0.79       211

    accuracy                           0.79       532
   macro avg       0.82      0.83      0.79       532
weighted avg       0.85      0.79      0.79       532

Train Confusion Matrix: 
 [[215 106]
 [  4 207]] 

Train Accuracy:  0.793233082706767


Classification Testing Report: 
               precision    recall  f1-score   support

           0       0.97      0.71      0.82       722
           1       0.70      0.97      0.81       522

    accuracy                           0.82      1244
   macro avg       0.84      0.84      0.82      1244
weighted avg       0.86      0.82      0.82      1244

Test Confusion Matrix: 
 [[511 211]
 [ 18 504]] 

Test Accuracy:  0.8159163987138264
